In [1]:
import sys
from numpy import array
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
sys.path.append('../src')
from keras.preprocessing.text import one_hot
from keras.models import load_model

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import parameters
from data_preprocess import loaddata, remove_impuritys, one_hot_encode, one_hot_decode, number_encode, predict_text

In [3]:
books=loaddata(parameters.datapath, 1)
books[:100]

"Harry Potter and the Sorcerer's Stone \n\nCHAPTER ONE \n\nTHE BOY WHO LIVED \n\nMr. and Mrs. Dursley, of n"

In [4]:
text=remove_impuritys(books)
books[:110]

"Harry Potter and the Sorcerer's Stone \n\nCHAPTER ONE \n\nTHE BOY WHO LIVED \n\nMr. and Mrs. Dursley, of number four"

In [5]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [6]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [7]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [8]:
## from keras.models import Sequential
from keras.layers import *

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=X_modified[0].shape,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences=True))
model.add(LSTM(256,return_sequences=True))
model.add(LSTM(256,return_sequences=True))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 200)          161600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 200)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 200)          320800    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 200)          320800    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 200)          320800    
_________________________________________________________________
lstm_5 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
__________

In [9]:
x_train=X_modified[:int(len(X_modified)*0.8)]
y_train=Y_modified[:int(len(Y_modified)*0.8)]
x_test=X_modified[int(len(X_modified)*0.8):]
y_test=Y_modified[int(len(Y_modified)*0.8):]

In [10]:
model.fit(x_train, y_train, epochs=1, batch_size=125, validation_split=0.1)


Train on 314337 samples, validate on 34927 samples
Epoch 1/1
 31625/314337 [==>...........................] - ETA: 34:19 - loss: 3.0473

KeyboardInterrupt: 

In [ ]:
model.save('my_model.h5')

In [ ]:
predict_text(model, X[99])